# Setup

TODO:
2. proveri na jednom modelu da li title poboljsava skor

In [ ]:
!pip install -q sentence-transformers;

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd drive/MyDrive/Colab Notebooks/siap/datasets

[Errno 2] No such file or directory: 'drive/MyDrive/Colab Notebooks/siap/datasets'
/content/drive/MyDrive/Colab Notebooks/siap/datasets


In [ ]:
from sklearn.metrics import classification_report
from sentence_transformers import SentenceTransformer
import tensorflow_hub as hub
import pandas as pd
from sklearn.metrics import f1_score
from sklearn import svm, neighbors
from enum import Enum

import warnings
warnings.filterwarnings("ignore")

In [ ]:
def get_classification_model(model_name):
  if model_name == 'svm linear':
    return svm.SVC(kernel='linear', random_state=1)
  elif model_name == 'svm rbf':
    return svm.SVC(kernel='rbf')
  elif model_name == 'knn':
    return neighbors.KNeighborsClassifier(n_neighbors=5)
  else:
    raise Exception('classification model not defined')

def get_embedding_model(model_name):
  return SentenceTransformer(model_name)

load models

In [ ]:
possible_train_datasets = ['train_clean_with_emoticons.csv', 'train_clean_removed_emoticons.csv', 'train_clean_replaced_emoticons.csv']
#possible_embedding_models = ['nli-mpnet-base-v2', 'all-mpnet-base-v2']
possible_embedding_models = ['nli-mpnet-base-v2']
#possible_clf_models = ['svm linear', 'svm rbf', 'knn']
possible_clf_models = ['svm linear']

In [ ]:
results = []

for train_dataset in possible_train_datasets:
  df_train = pd.read_csv(train_dataset)
  df_validation = pd.read_csv(train_dataset.replace('train', 'validation'))
  y_train = df_train['Rating']
  y_validation = df_validation['Rating']

  for embedding_model_name in possible_embedding_models: 
    embedding_model = get_embedding_model(embedding_model_name)
    X_train = embedding_model.encode(list(df_train['Review Text'].values))
    X_validation = embedding_model.encode(list(df_validation['Review Text'].values))
    
    for clf_model_name in possible_clf_models:
      clf_model = get_classification_model(clf_model_name)
      clf_model.fit(X_train, y_train)
      
      y_train_pred = clf_model.predict(X_train)
      y_validation_pred = clf_model.predict(X_validation)

      f1_train = f1_score(y_train, y_train_pred, average='micro')
      f1_validation = f1_score(y_validation, y_validation_pred, average='micro')

      results.append((train_dataset, embedding_model_name, clf_model_name, f1_train, f1_validation))


In [ ]:
sorted_results = sorted(results, key=lambda x: x[4], reverse=True)
for res in sorted_results:
  print(res)

('train_clean_removed_emoticons.csv', 'nli-mpnet-base-v2', 'svm linear', 0.7082758239938166, 0.6737400530503979)
('train_clean_with_emoticons.csv', 'nli-mpnet-base-v2', 'svm linear', 0.7086622867553689, 0.6732979664014147)
('train_clean_replaced_emoticons.csv', 'nli-mpnet-base-v2', 'svm linear', 0.7080549881300724, 0.6724137931034483)
